In [ ]:
import cv2 as cv
from matplotlib.pyplot import imshow, show, figure
import imutils
import numpy as np

def cv2_imshow(image, figsize=(9,9)):
    fig = figure(figsize=figsize)
    try:
        imshow(image[:,:,::-1])
    except IndexError:
        imshow(image, cmap="gray")
    show()

In [ ]:
def extract_corners(filename):
    kernel = np.ones((3,3),np.uint8)
    #Read original image and grayscale
    img_original = cv.imread(filename)
    img = cv.cvtColor(cv.imread(filename), cv.COLOR_BGR2GRAY)
    img_copy = img.copy()
    
    
    #Transform in black and white
    imgt = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,57,3)
    
    imgt_copy = imgt.copy()
    #Find external countour, create mask and draw external countour on mask
    contours, hierarchy = cv.findContours(image=imgt_copy, mode=cv.RETR_EXTERNAL, method=cv.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(imgt)
    cv2_imshow(imgt_copy)
    max_c = 0
    max_area = 0
    for contorno in contours:
        c_area = cv.contourArea(contorno)
        if c_area > max_area:
            max_area = c_area
            max_c = contorno
    cv.drawContours(image=mask, contours=[max_c], contourIdx=-1, color=(255), thickness=5, lineType=cv.LINE_4)
    cv2_imshow(mask)
    corners = []
    for i in np.linspace(1, 0, 20): # Busca de melhor parâmetro
        #Find grid corners and draw on original image
        corners = cv.goodFeaturesToTrack(mask, 4, i, 50)
        if corners is None: continue
        for corner in corners:
            x,y = corner.ravel().astype(int)
        corners = sorted(corners.copy().astype(np.float32).reshape(corners.shape[0], corners.shape[2]), key=lambda x: x[0])
        if len(corners) == 4:
            break
    # Magical for-else loop, only enters else if finish loop without break
    else:
        raise Exception("Unable to find corners for supplied image! Are you sure it's a valid sudoku?")
    pts1 = []
    a,b,c,d = corners
    if a[1]>b[1]:
        pts1.append(list(a))
        pts1.append(list(b))
    else:
        pts1.append(list(b))
        pts1.append(list(a))
    if c[1]>d[1]:
        pts1.append(list(c))
        pts1.append(list(d))
    else:
        pts1.append(list(d))
        pts1.append(list(c))
    # Images for debugging
    # cv2_imshow(img_original)
    # cv2_imshow(imgt)
    # cv2_imshow(mask)
    # pts1 = Bot-Left; Top-Left; Bot-Right; Top-Right
    pts1 = np.array(pts1)
    return pts1

def produce_transform(filename: str, output_size: int):
    pts1 = extract_corners(filename)
    img_clean = cv.imread(filename)
    pts2 = np.float32([[0,output_size], [0,0], [output_size,output_size], [output_size,0]])
    # Draw circles on output, disabled for production, useful for debugging
    # for val in pts1:
    #     cv.circle(img_clean, [int(val[0]),int(val[1])], 1, (0,255,0), -1)
    M = cv.getPerspectiveTransform(pts1,pts2)
    dst = cv.warpPerspective(img_clean,M,(output_size,output_size))
    cv2_imshow(dst)
    return dst.copy()

def unsharp_mask(image, kernel_size=(7, 7), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

def adaptive_threshold_cleanup(dst):
    clahe = cv.createCLAHE(clipLimit=3.0, tileGridSize=(11, 11))
    equalized = clahe.apply(cv.cvtColor(dst, cv.COLOR_BGR2GRAY))
    sharp = unsharp_mask(equalized)
    imgt = cv.adaptiveThreshold(sharp ,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,51,31)
    d = cv.dilate(imgt, (3,3))
    p = cv.erode(d, (3,3))
    return ~p

def show_all_cells(img):
    h, w = img.shape
    cell_size = int(h/9)
    min_x = 0
    min_y = 0
    for i in range(9):
        for j in range(9):
            cell_img = img[min_x:(min_x+cell_size), min_y:(min_y + cell_size)]
            cv2_imshow(cell_img)
            min_x += cell_size
        min_x = 0
        min_y += cell_size


In [ ]:
result = adaptive_threshold_cleanup(produce_transform('image1062.jpg', 9*28))
cv2_imshow(result)

In [ ]:
show_all_cells(result)